### Imports and Setup

In [1]:
import cv2
import numpy as np
from keras.models import load_model
from collections import deque
import pickle

### Load Model and Label Binarizer


In [2]:
# Load your model
model = load_model(r"C:\Rajat\COLLEGE\Semester 7\Hyperverge\football_openCV\model\videoclassificationmodel")

# Load the label binarizer
with open(r"C:\Rajat\COLLEGE\Semester 7\Hyperverge\football_openCV\model\videoclassification.pickle", "rb") as f:
    lb = pickle.load(f)

### Setup Video Processing

In [3]:
outputvideo = r"C:\Rajat\COLLEGE\Semester 7\Hyperverge\football_openCV\model\demo_output.mp4"
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
Queue = deque(maxlen=128)

capture_video = cv2.VideoCapture(r"C:\Rajat\COLLEGE\Semester 7\Hyperverge\football_openCV\model\tennis.mp4")
writer = None
(Width, Height) = (None, None)
Sports_labels = ['boxing', 'swimming', 'tennis']


### Process Video Frame-by-Frame


In [5]:
outputvideo = r"C:\Rajat\COLLEGE\Semester 7\Hyperverge\football_openCV\model\demo_output.mp4"
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
Queue = deque(maxlen=128)

# Open video capture
capture_video = cv2.VideoCapture(r"C:\Rajat\COLLEGE\Semester 7\Hyperverge\football_openCV\model\tennis.mp4")
writer = None
(Width, Height) = (None, None)
Sports_labels = ['boxing', 'swimming', 'tennis']
while True:
    (taken, Frame) = capture_video.read()
    if not taken:
        break
    if Width is None or Height is None:
        (Width, Height) = Frame.shape[:2]
    output = Frame.copy()
    frame = cv2.cvtColor(Frame, cv2.COLOR_BGR2RGB)
    
    # Resize to the expected input shape for the model
    frame = cv2.resize(frame, (244, 224)).astype("float32")
    frame -= mean
    
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    Queue.append(preds)
    results = np.array(Queue).mean(axis=0)
    i = np.argmax(results)
    label = Sports_labels[lb.classes_[i]]
    text = "They are Playing: {}".format(label)
    # cv2.putText(output, text, (45, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (255, 255, 255), 5)
    # Draw the text with a black border
    cv2.putText(output, text, (45, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 0, 0), 7)
    # Draw the text with white color
    cv2.putText(output, text, (45, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (255, 255, 255), 5)

    
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(outputvideo, fourcc, 30, (Width, Height), True)
    
    writer.write(output)
    cv2.imshow("In progress", output)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

print("Finalizing.....")
writer.release()
capture_video.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 176ms/step
Finalizing.....
